# Implementarea unui Sistem Expert pentru Animale

Un exemplu din [Curriculum AI pentru Începători](http://github.com/microsoft/ai-for-beginners).

În acest exemplu, vom implementa un sistem simplu bazat pe cunoștințe pentru a determina un animal pe baza unor caracteristici fizice. Sistemul poate fi reprezentat prin următorul arbore AND-OR (aceasta este o parte a întregului arbore, putem adăuga cu ușurință mai multe reguli):

![](../../../../../../translated_images/ro/AND-OR-Tree.5592d2c70187f283.webp)


## Propria noastră interfață expert pentru sisteme cu inferență inversă

Să încercăm să definim un limbaj simplu pentru reprezentarea cunoștințelor bazat pe reguli de producție. Vom folosi clase Python ca și cuvinte cheie pentru a defini regulile. Ar exista în esență 3 tipuri de clase:
* `Ask` reprezintă o întrebare care trebuie adresată utilizatorului. Conține setul de posibile răspunsuri.
* `If` reprezintă o regulă și este doar un zahăr sintactic pentru a stoca conținutul regulii.
* `AND`/`OR` sunt clase pentru a reprezenta ramurile AND/OR ale arborelui. Acestea stochează doar lista de argumente în interior. Pentru a simplifica codul, toată funcționalitatea este definită în clasa părinte `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

În sistemul nostru, memoria de lucru ar conține lista de **fapte** ca **perechi atribut-valoare**. Baza de cunoștințe poate fi definită ca un singur dicționar mare care asociază acțiunile (noi fapte ce trebuie inserate în memoria de lucru) cu condiții, exprimate ca expresii AND-OR. De asemenea, unele fapte pot fi `Ask`-uite.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Pentru a efectua inferența inversă, vom defini clasa `Knowledgebase`. Aceasta va conține:
* Memoria de lucru `memory` - un dicționar care asociază atributele cu valorile
* Regulile din baza de cunoștințe `rules` în formatul definit mai sus

Două metode principale sunt:
* `get` pentru a obține valoarea unui atribut, efectuând inferența dacă este necesar. De exemplu, `get('color')` ar obține valoarea unui slot de culoare (va întreba dacă este necesar și va stoca valoarea pentru utilizări ulterioare în memoria de lucru). Dacă întrebăm `get('color:blue')`, va întreba pentru o culoare și apoi va returna valoarea `y`/`n` în funcție de culoare.
* `eval` realizează inferența propriu-zisă, adică parcurge arborele AND/OR, evaluează sub-obiectivele etc.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Acum să definim baza noastră de cunoștințe despre animale și să efectuăm consultația. Rețineți că acest apel vă va pune întrebări. Puteți răspunde tastând `y`/`n` pentru întrebări de tip da-nu, sau specificând un număr (0..N) pentru întrebări cu răspunsuri multiple mai lungi.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Folosirea Experta pentru Inferență Directă

În următorul exemplu, vom încerca să implementăm inferența directă folosind una dintre bibliotecile pentru reprezentarea cunoștințelor, [Experta](https://github.com/nilp0inter/experta). **Experta** este o bibliotecă pentru crearea de sisteme de inferență directă în Python, concepută să fie similară cu sistemul clasic vechi [CLIPS](http://www.clipsrules.net/index.html).

Am fi putut implementa și noi lanțuirea înainte fără multe probleme, dar implementările naive de obicei nu sunt foarte eficiente. Pentru potrivirea mai eficientă a regulilor se folosește un algoritm special [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Vom defini sistemul nostru ca o clasă care derivă din `KnowledgeEngine`. Fiecare regulă este definită printr-o funcție separată cu adnotarea `@Rule`, care specifică când ar trebui să se declanșeze regula. În interiorul regulii, putem adăuga noi fapte folosind funcția `declare`, iar adăugarea acestor fapte va determina apelarea unor reguli suplimentare de către motorul de inferență înainte.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Odată ce am definit o bază de cunoștințe, populăm memoria de lucru cu câteva fapte inițiale, apoi apelăm metoda `run()` pentru a efectua inferența. Putem observa ca rezultat că noi fapte deduse sunt adăugate în memoria de lucru, inclusiv faptul final despre animal (dacă am configurat toate faptele inițiale corect).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși depunem eforturi pentru a asigura acuratețea, vă rugăm să țineți cont că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa nativă trebuie considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm răspunderea pentru eventuale neînțelegeri sau interpretări greșite rezultate din utilizarea acestei traduceri.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
